In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from solcore.solar_cell import SolarCell, default_GaAs
from solcore.structure import Layer, Junction
from solcore import si
from solcore import material
from solcore.light_source import LightSource
from solcore.solar_cell_solver import solar_cell_solver

TODO The purpose of this example needs to be clarified. It is messy

In [ ]:
T = 298

In [ ]:
substrate = material('GaAs')(T=T)

In [ ]:
def AlGaAs(T):
    # We create the other materials we need for the device
    window = material('AlGaAs')(T=T, Na=5e24, Al=0.8)
    p_AlGaAs = material('AlGaAs')(T=T, Na=1e24, Al=0.4)
    n_AlGaAs = material('AlGaAs')(T=T, Nd=8e22, Al=0.4)
    bsf = material('AlGaAs')(T=T, Nd=2e24, Al=0.6)
    output = Junction([Layer(width=si('30nm'), material=window, role="Window"),
                       Layer(width=si('150nm'), material=p_AlGaAs, role="Emitter"),
                       Layer(width=si('1000nm'), material=n_AlGaAs, role="Base"),
                       Layer(width=si('200nm'), material=bsf, role="BSF")], sn=1e6, sp=1e6, T=T, kind='PDD')
    return output

In [ ]:
my_solar_cell = SolarCell([AlGaAs(T), default_GaAs(T)], T=T, R_series=0, substrate=substrate)

In [ ]:
Vin = np.linspace(-2, 2.61, 201)
V = np.linspace(0, 2.6, 300)
wl = np.linspace(350, 1000, 301) * 1e-9
light_source = LightSource(source_type='standard', version='AM1.5g', x=wl, output_units='photon_flux_per_m',
                           concentration=1)

In [ ]:
solar_cell_solver(my_solar_cell, 'equilibrium')

plt.figure(1)
for j in my_solar_cell.junction_indices:
    zz = my_solar_cell[j].equilibrium_data.Properties['x'] + my_solar_cell[j].offset
    n = my_solar_cell[j].equilibrium_data.Properties['Nd']
    p = my_solar_cell[j].equilibrium_data.Properties['Na']
    plt.semilogy(zz, n, 'b')
    plt.semilogy(zz, p, 'r')

    zz = my_solar_cell[j].equilibrium_data.Bandstructure['x'] + my_solar_cell[j].offset
    n = my_solar_cell[j].equilibrium_data.Bandstructure['n']
    p = my_solar_cell[j].equilibrium_data.Bandstructure['p']
    plt.semilogy(zz, n, 'b--')
    plt.semilogy(zz, p, 'r--')

plt.xlabel('Position (m)')
plt.ylabel('Carrier density (m$^{-3}$)')

plt.show()